In [1]:
%reload_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [4]:
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/kabirkhan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
!spacy download en_core_web_sm

    100% |████████████████████████████████| 12.1MB 52.6MB/s ta 0:00:01
  Running setup.py install for en-core-web-sm ... done
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
from types import SimpleNamespace
from typing import Any, Dict, List, Optional

from recon.dataset import Dataset
from pydantic import root_validator
from recon.types import Example, Span, Token
import numpy as np
from recon.augmentation import augment_example
from recon.operations import operation, registry
from recon.preprocess import SpacyPreProcessor

In [ ]:
@operation("recon.v1.augment.verb_replace", factory=True)
def kb_expansion(
    example: Example,
    preprocessed_outputs: Dict[str, Any] = {},
    n_augs: int = 1,
    sub_prob: float = 0.5,
) -> List[Example]:

    spans_to_aliases_map = preprocessed_outputs["recon.v1.span_aliases"]
    print(preprocessed_outputs)
    print(example.data)

    def augmentation(span: Span, spans_to_aliases_map: Dict[int, List[str]]) -> Optional[str]:
        if hash(span) in spans_to_aliases_map:
            aliases = spans_to_aliases_map[hash(span)]

            if len(aliases) > 0:
                rand_alias = np.random.choice(aliases)
                index = aliases.index(rand_alias)
                del spans_to_aliases_map[hash(span)][index]
                return rand_alias

    return augment_example(
        example,
        augmentation,
        n_augs=n_augs,
        sub_prob=sub_prob,
        spans_to_aliases_map=spans_to_aliases_map,
    )

In [182]:
def substitute_spans(example: Example, span_subs: Dict[Span, str]) -> Example:
    """Substitute spans in an example. Replaces span text and alters the example text
    and span offsets to create a valid example.

    Args:
        example (Example): Input example
        span_subs (Dict[int, str]): Mapping of span hash to a str replacement text

    Returns:
        Example: Output example with substituted spans
    """
    span_sub_start_counter = 0

    new_example_text = example.text
    new_example_spans = []
    
    prev_example_spans = {hash(span) for span in example.spans}
    spans = sorted(list(span_subs.keys()) + example.spans, key=lambda s: s.start)
    
    for span in spans:
        should_add_span = hash(span) in prev_example_spans
        
        prev_end = span.end
        new_text = span.text

        if span in span_subs:
            new_text = span_subs[span]
            new_start = span.start + span_sub_start_counter
            new_end = new_start + len(new_text)

            new_example_text = (
                new_example_text[: span.start + span_sub_start_counter]
                + new_text
                + new_example_text[span.end + span_sub_start_counter :]
            )

            span.text = new_text
            span.start = new_start
            span.end = new_end
            
            span_sub_start_counter += new_end - prev_end
        else:
            span.start += span_sub_start_counter
            span.end = span.start + len(new_text)
            span_sub_start_counter = span.end - prev_end

        span.text = new_text
        
        if should_add_span:
            new_example_spans.append(span)
        
    example.text = new_example_text
    example.spans = new_example_spans

    return example


In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")
spacy_pre = SpacyPreProcessor(nlp)

In [16]:
[t.pos_ for t in doc]

NameError: name 'doc' is not defined

In [ ]:
import nltk
from nltk.corpus import wordnet as wn

def get_synonym(word, pos=None):
    """Get synonym for word given its part-of-speech (pos)."""
    synsets = wn.synsets(word, pos=pos)
    # Return None if wordnet has no synsets (synonym sets) for this word and pos.
    if synsets:
        print(synsets[0])
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        print("SYNSET WORDS: ", words)
        if words[0].lower() != word.lower():  # Skip if synonym is same as word.
            # Multi word synonyms in wordnet use '_' as a separator e.g. reckon_with. Replace it with space.
            return words[0].replace("_", " ")


def replace_token(spacy_doc, idx, replacement):
    """Replace token in position idx with replacement."""
    return " ".join([spacy_doc[:idx].text, replacement, spacy_doc[1 + idx :].text])

In [17]:
@operation("recon.v1.augment.replace_pos_with_synonym", pre=[spacy_pre])
def replace_pos_with_synonym(example: Example, pos: str, preprocessed_outputs={}, n_augs: int = 1):
    
    prev_example = example.copy(deep=True)
    augmented_examples = {prev_example}
    
    pos_map = {
        "VERB": "v",
        "NOUN": "n",
        "ADJ": "a"
    }
    
    if pos not in pos_map:
        raise ValueError(f"Argument `pos` of {pos} not in {''.join(pos_map.keys())}")
    
    for i in range(n_augs):
        example = example.copy(deep=True)
        doc = example.data.doc
        span_starts = [s.start for s in example.spans]

        # Get indices of verb tokens in sentence.
        pos_idxs = [i for i, token in enumerate(doc) if token.pos_ == pos and token.idx not in span_starts]
        if pos_idxs:
            # Pick random verb idx to replace.
            idx = np.random.choice(pos_idxs)
            token = doc[idx]
            synonym = get_synonym(token.text, pos="v")
            synonym = "enjoy"
            print(synonym)
            
            
            # If there's a valid verb synonym, replace it. Otherwise, return None.
            if synonym:
                curr_span = Span(text=token.text, start=token.idx, end=token.idx + len(token.text), label="")
                span_subs = {curr_span: synonym}
                example = substitute_spans(example, span_subs)
                if example not in augmented_examples:
                    augmented_examples.add(example)
                    
    return list(augmented_examples)

In [19]:
examples = [Example(text="I love running.", spans=[Span(text="running", start=7, end=14, label="ACTIVITY")])]
docs = spacy_pre(examples)

for e, doc in zip(examples, docs):
    e.doc = doc

ds = Dataset("test", data=examples)

# ds.apply_("recon.v1.augment.replace_pos_with_synonym", "VERB")

# ds.data

In [201]:
replace_adjective_with_synonym(examples[0])

[1, 2]
Synset('love.v.01')
SYNSET WORDS:  ['love']
enjoy


Example(text='I enjoy running.', spans=[Span(text='running', start=8, end=15, label='ACTIVITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True, data=namespace(doc=I love running.))

In [189]:
examples[0]

Example(text='I enjoy running.', spans=[Span(text='running', start=8, end=15, label='ACTIVITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True, data=namespace(doc=I love running.))

In [210]:
!pip install snorkel

[autoreload of recon.augmentation failed: Traceback (most recent call last):
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/kabirkhan/Docume

    100% |████████████████████████████████| 143kB 2.5MB/s ta 0:00:01
    100% |████████████████████████████████| 748.8MB 2.2kB/s eta 0:00:011    94% |██████████████████████████████▍ | 710.1MB 16.6MB/s eta 0:00:03    97% |███████████████████████████████▏| 728.1MB 27.7MB/s eta 0:00:01
    100% |████████████████████████████████| 3.8MB 509kB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 1.8MB 1.1MB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 798kB 2.0MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 4.0MB/s eta 0:00:01
    100% |████████████████████████████████| 19.7MB 91kB/s  eta 0:00:011
    100% |████████

In [4]:
!pip install names

    100% |████████████████████████████████| 798kB 1.6MB/s eta 0:00:01
  Running setup.py bdist_wheel for names ... done
  Stored in directory: /home/kabirkhan/.cache/pip/wheels/f9/a5/e1/be3e0aaa6fa285575078fa2aafd9959b45bdbc8de8a6803aeb
Successfully built names


In [1]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [5]:
import names
from snorkel.augmentation import transformation_function

# Pregenerate some random person names to replace existing ones with
# for the transformation strategies below
replacement_names = [names.get_full_name() for _ in range(50)]


# Replace a random named entity with a different entity of the same type.
@transformation_function(pre=[spacy])
def change_person(x):
    person_names = [ent.text for ent in x.doc.ents if ent.label_ == "PERSON"]
    # If there is at least one person name, replace a random one. Else return None.
    if person_names:
        name_to_replace = np.random.choice(person_names)
        replacement_name = np.random.choice(replacement_names)
        x.text = x.text.replace(name_to_replace, replacement_name)
        return x


# Swap two adjectives at random.
@transformation_function(pre=[spacy])
def swap_adjectives(x):
    adjective_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "ADJ"]
    # Check that there are at least two adjectives to swap.
    if len(adjective_idxs) >= 2:
        idx1, idx2 = sorted(np.random.choice(adjective_idxs, 2, replace=False))
        # Swap tokens in positions idx1 and idx2.
        x.text = " ".join(
            [
                x.doc[:idx1].text,
                x.doc[idx2].text,
                x.doc[1 + idx1 : idx2].text,
                x.doc[idx1].text,
                x.doc[1 + idx2 :].text,
            ]
        )
        return x

In [11]:

@operation("recon.v1.augment.ent_label_sub", handles_tokens=False)
def ent_label_sub(
    example: Example, label: str, subs: List[str], n_augs: int = 1, sub_prob: float = 0.5
) -> List[Example]:
    """Augmentation to substitute entities based on label.
    Applies a mask to the entities that have a provided label based on substitution_prob
    and selects a random choice from the list of provided substitutions to replace each
    span with

    Args:
        example (Example): Input example
        label (str): Span label to replace
            e.g. PERSON or LOCATION
        subs (List[str]): List of substitutions
            e.g. list of names if label is PERSON
        n_augs (int, optional): Maximum number of augmentated examples to create per example.
        sub_prob (float, optional): Probability from 0-1 inclusive of how many of the spans to replace

    Returns:
        List[Example]: List of augmented examples including the original.
    """

    def augmentation(span: Span, subs: List[str]) -> Optional[str]:
        subs = [s for s in subs if s != span.text]
        sub = None
        if len(subs) > 0:
            sub = np.random.choice(subs)
        return sub

    return augment_example(example, augmentation, n_augs=n_augs, sub_prob=sub_prob, subs=subs)

In [33]:
from typing import Callable

In [37]:
def make_transformation_function(
    spans: List[Span],
    span_f: Callable[[Span, Any], Optional[str]],
    span_label: str = None,
    sub_prob: float = 0.5,
    **kwargs: Any
):
    if span_label:
        spans = [s for s in spans if s.label == span_label]
    mask = mask_1d(len(spans), prob=sub_prob)
    spans_to_sub = list(np.asarray(spans)[mask])
    
    
    span = np.random.choice(spans_to_sub)
    res = span_f(span, **kwargs)
    return res


In [38]:
def ent_label_sub_augmentation(span: Span, subs: List[str]) -> Optional[str]:
    subs = [s for s in subs if s != span.text]
    sub = None
    if len(subs) > 0:
        sub = np.random.choice(subs)
    return sub

In [40]:
ds.data[0]

Example(text='I love running.', spans=[Span(text='running', start=7, end=14, label='ACTIVITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True, doc=I love running.)

In [ ]:
make_transformation_function()

In [68]:
make_transformation_function

<function __main__.make_transformation_function(spans:List[recon.types.Span], span_f:Callable[[recon.types.Span, Any], Union[str, NoneType]], span_label:str=None, sub_prob:float=0.5, **kwargs:Any)>

In [67]:
def augmentation(span: Span, subs: List[str]) -> Optional[str]:
    subs = [s for s in subs if s != span.text]
    sub = None
    if len(subs) > 0:
        sub = np.random.choice(subs)
    return sub

NameError: name 'augmentation' is not defined

In [ ]:
def augment_example(
    example: Example,
    augmentation_f: Callable[[Span, Any], Optional[str]],
    spans: List[Span] = None,
    span_label: str = None,
    n_augs: int = 1,
    sub_prob: float = 0.5,
    **kwargs: Any,
) -> Optional[Example]:
    

    if spans is None:
        spans = example.spans

    prev_example = example.copy(deep=True)
    if span_label:
        spans = [s for s in spans if s.label == span_label]
    mask = mask_1d(len(spans), prob=sub_prob)
    spans_to_sub = list(np.asarray(spans)[mask])

    span_subs = {}
    for span in spans_to_sub:
        res = span_f(span, **kwargs)  #  type: ignore
        if res:
            span_subs[hash(span)] = res

    if not any(span_subs.values()) or len(augmented_examples) > n_augs:
        break

    example = substitute_spans(example, span_subs)
    if hash(example) != hash(prev_example):
        return example

    return list(augmented_examples)

In [20]:
ds.apply_(person_ent_sub)

ValueError: This function is not an operation since it does not have a name.

In [8]:
tfs = [change_person, swap_adjectives]

In [9]:
from snorkel.augmentation import RandomPolicy

random_policy = RandomPolicy(
    len(tfs), sequence_length=2, n_per_original=2, keep_original=True
)

In [66]:
random_policy.generate_for_example()

[[], [0, 0], [0, 1]]

In [6]:
from recon.augmentation import substitute_spans, augment_example

In [ ]:
from tqdm import tqdm

from snorkel.augmentation.core import BaseTFApplier


class ReconDatasetTFApplier(BaseTFApplier):
    
    def __init__(self, tfs, policy, span_label: str, sub_prob: float = 0.5):
        super().__init__(tfs, policy)
        self.span_label = span_label
        self.sub_prob = sub_prob
    
    def _apply_policy_to_data_point(self, x: DataPoint) -> DataPoints:
        
        x_transformed = []
        for seq in self._policy.generate_for_example():
            x_t = x
            # Handle empty sequence for `keep_original`
            transform_applied = len(seq) == 0
            # Apply TFs
            for tf_idx in seq:
                
                
                if spans is None:
                    spans = example.spans

                prev_example = x.copy(deep=True)
                if self.span_label:
                    spans = [s for s in spans if s.label == self.span_label]
                mask = mask_1d(len(spans), prob=sub_prob)
                spans_to_sub = list(np.asarray(spans)[mask])

                span_subs = {}
                tf = self._tfs[tf_idx]
                for span in spans_to_sub:
                    x_t_or_none = tf(span, **kwargs)  #  type: ignore
                    if x_t_or_none is not None:
                        transform_applied = True
                        span_subs[hash(span)] = res

                
                x_t_or_none = tf(x_t)
                # Update if transformation was applied
                if x_t_or_none is not None:
                    transform_applied = True
                    x_t = x_t_or_none
            # Add example if original or transformations applied
            if transform_applied:
                x_transformed.append(x_t)
        return x_transformed


    def apply(self, ds: Dataset, progress_bar: bool = True) -> Dataset:
        x_transformed: List[Example] = []
            
        for _, example in tqdm(ds.data, total=len(df), disable=(not progress_bar)):
            
            transformed_examples = self._apply_policy_to_data_point(x)
            
            x_transformed.extend(examples_transformed)
        return pd.concat(x_transformed, axis=1).T.infer_objects()

In [ ]:
applier = ReconDatasetTFApplier([])